In [ ]:
import numpy as np
import os as os
import tifffile as tif
import cv2
from loadmat import loadmat_file as load
from remove_single_part import remove_single_part as rsp
from padding import padding as padding    
from create_fields import create_fields as fields
from calc_fields_out import calc_fields as calc
from binning import binning
from remove_isolated_px import remove_isolated_px as mask
from smoothing import smoothing
from SaveInputParameter import SaveInputShortTrAn

In [ ]:
Num = 20 #number of datasets to be evaluated
input_path = '/Volumes/Vin/Python/shortTrAn/Trc_' #without '.mat'
resultpath = '/Volumes/Vin/Python/shortTrAn/results'
time_lag = 10 #timelag ms
time_exp = 9 #time expose in ms
scale = 160 #pixelation of camera (160 nm) or pixelation of simulation mask (10 nm) 
sigma = 26 #localisation precision in nm
errcorr = 'No' #shall be error correction conducted? 'Yes' or 'No'


scal = 160 #binning to px of 160 nm


#Parameter for Filter1: Removal of pixels containing less entries than the threshold.
threshold = 6 

#Parameter for Filter2: Removal of isolated pixels.
kdim = 3 #odd number, defines folding kernel . Largest number for cross kernel is 5.
kshape='box' #cross or box (kernel). 

#Parameter for Filter3: Smoothing out.
kdim_smoothing= 3 #odd number defines smoothing kernel.
wdth_outliers = 2 #defines width of outlier.
mode_outliers = 'gauss' #or 'remove'

In [ ]:
SaveInputShortTrAn(input_path, resultpath, time_lag, time_exp, scale,sigma,
                       errcorr, scal, threshold, kdim, kshape, kdim_smoothing,
                       wdth_outliers, mode_outliers)

for N in range(1,Num+1):
    
    # LOADING AND PREPARTION OF THE DATA SETS FOR ANALYSIS
    
    path = (input_path + str(N))
    resultpath = resultpath
    
    #Imports data and applies the unit transfomation to 1 nm per pixel.
    a, b, tracs,tracs_unint16, trac_num, counts, tracs_unint16corr, trac_numcorr, dt = load(path,scale)

    #Removal of all trajectories of an length shorter than 2 frames.
    tracs_unint16corr, trac_numcorr = rsp(trac_num,counts,a,tracs_unint16corr,trac_numcorr)

    #Padding. Creates additional space which is later used for the downscaling/binning.
    x_min, x_max, y_min, y_max, a, tracs_unint16corr, x_length, y_length, shape\
    = padding(tracs_unint16corr, a, resultpath, N)
    #Creation of multiple feature space containing the countfield, diffusiontensor(xx,xy,yx,yy) 
    #and directed motion (x,y).
    pointfield,vectorfield_x,vectorfield_y,tensorfield_xx,tensorfield_xy,tensorfield_yx,tensorfield_yy=fields(shape)
    
    
    #CALCULATION OF THE LOCAL DIFFUSION COEFFICIENTS AND SPEEDS
    
    #Calculation of the derivates for each position of a trajectory based on the positional information of 
    #the input data.
    #Followed by the calculation of the directed motion and the tranfer of the pixel wise sum of the counts,  
    #diffusion coefficients and the speeds in the multilayer feature space.
    directory,tensorfield_xx,tensorfield_xy,tensorfield_yx,tensorfield_yy,vectorfield_x,vectorfield_y\
    =calc(a,time_lag,sigma,time_exp,path,resultpath,N,errcorr,pointfield,tracs_unint16corr,
                 trac_numcorr,vectorfield_x,vectorfield_y,tensorfield_xx,tensorfield_xy,tensorfield_yx,tensorfield_yy)
    
    
    #BINNING INTO LARGER LOCAL SQUARES
    
    #where to save
    directory = resultpath + '/Trc' + str(N)
    #directory = path_input + str(num) + '/Binning_200'
    
    #Downscaling/binning. The entries of the new pixelation contain the sum of the underlying pixelation which is 
    #is normalized by the number of contributing pixels.
    scaled_pointfield,scaled_pointfield_inv,scaled_vectorfield_x,scaled_vectorfield_y,scaled_tensorfield_xx,\
    scaled_tensorfield_xy,scaled_tensorfield_yx,scaled_tensorfield_yy,scld_ten_xx,scld_ten_xy,scld_ten_yx,\
    scld_ten_yy,scld_vec_x,scld_vec_y=binning(scal,directory,pointfield,vectorfield_x,vectorfield_y,
                                              tensorfield_xx,tensorfield_xy,tensorfield_yx,tensorfield_yy)
    
    
    #APLLICATION OF SPATIAL FILTER SEQUENCE
    
    if not os.path.exists(directory+'/results_filtering'):
        #print('Path does not exist')
        os.mkdir(directory+'/results_filtering')
        directory = directory+'/results_filtering/'
    else:
        directory = directory+'/results_filtering/'

    #Filter #1 and #2. Removes all pixels which have less entries then threshold x and are isolated.
    scld_count_mskd,scld_ten_xx_mskd,scld_ten_xy_mskd,scld_ten_yx_mskd,scld_ten_yy_mskd,scld_vec_x_mskd, \
        scld_vec_y_mskd=mask(kdim,kshape,threshold,directory,scaled_pointfield,scld_ten_xx,
                                           scld_ten_xy,scld_ten_yx,scld_ten_yy,scld_vec_x,scld_vec_y)

    #Filter #3. Smoothes entries to the surrounding pixels using a for contributing entries weighted average filter 
    #of an certain kernel size kdim.
    scld_count_fil,scld_vec_x_fil,scld_vec_y_fil,scld_ten_xx_fil,scld_ten_xy_fil,scld_ten_yx_fil,scld_ten_yy_fil\
    =smoothing(kdim_smoothing,wdth_outliers,mode_outliers,directory,scaled_pointfield,scld_count_mskd,scld_vec_x_mskd, 
               scld_vec_y_mskd,scld_ten_xx_mskd,scld_ten_xy_mskd,scld_ten_yx_mskd,scld_ten_yy_mskd)

    #Saves input parameter in text file:
    txt = open(directory +'input_parameter.txt',"w+")
    txt.write(directory+'N\n')
    txt.write('Number of datasets: '+str(N) + '\n')
    txt.write('Filter 1: Threshold '+str(threshold) + '\n') 
    txt.write('Filter 2: Kernel dimension '+str(kdim) + ' and shape ' + kshape + ' kernel \n')  
    txt.write('Filter 3: Kernel size '+str(kdim_smoothing)+', width outlier '+str(wdth_outliers)+', '+mode_outliers)
    txt.close()
    
    print('Finished cell ' +str(N))